In [1]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from ddn3 import simulation
from ddn3_extra import simulation_r
from iddn_paper import sim3_h5op, tool_sys


In [2]:
graph_type = "hub"
# graph_type = "scale-free"
n_sample_gen = 10000
n_node = 100
ratio_diff = 0.25
n_group = 5  # 5
idx = np.random.randint(1000000)
f_out = (
    f"sim3_ggm_{graph_type}_n-node_{n_node}_n-group_{n_group}_ratio_{ratio_diff}_{idx}"
)
print(f_out)
dat_file = tool_sys.get_work_folder() + f"/experiment_iddn_paper/sim_input/{f_out}.hdf5"

omega, omega1, omega2 = simulation_r.huge_omega(
    n_node=n_node,
    ratio_diff=ratio_diff,
    graph_type=graph_type,
    n_group=n_group,
)
g1_cov, g2_cov, comm_gt, diff_gt = simulation.prep_sim_from_two_omega(omega1, omega2)
con_mat1 = 1 * (np.abs(omega1) > 1e-8)
con_mat2 = 1 * (np.abs(omega2) > 1e-8)
con_mat1[np.arange(n_node), np.arange(n_node)] = 0
con_mat2[np.arange(n_node), np.arange(n_node)] = 0

n1, n2 = n_sample_gen, n_sample_gen
dat1, dat2 = simulation.gen_sample_two_conditions(g1_cov, g2_cov, n1, n2)


sim3_ggm_hub_n-node_100_n-group_5_ratio_0.25_595341


In [6]:
# plt.imshow(g1_cov)
# plt.clim([-1,1])
# plt.colorbar()

In [43]:
dep_mat = np.ones((n_node, n_node))
dep_mat_null = dep_mat
# dep_mat_prior = dep_mat

msk_mat = comm_gt+diff_gt
idx0, idx1 = np.where(msk_mat == 0)
n_zero = len(idx0)

dep_mat_prior = np.copy(msk_mat)
idx_sel = np.random.choice(n_zero, int(n_zero*0.05))
idx0_sel = idx0[idx_sel]
idx1_sel = idx1[idx_sel]
dep_mat_prior[idx0_sel, idx1_sel] = 1
dep_mat_prior[idx1_sel, idx0_sel] = 1

dep_mat_prior_loose = np.copy(msk_mat)
idx_sel = np.random.choice(n_zero, int(n_zero*0.25))
idx0_sel = idx0[idx_sel]
idx1_sel = idx1[idx_sel]
dep_mat_prior_loose[idx0_sel, idx1_sel] = 1
dep_mat_prior_loose[idx1_sel, idx0_sel] = 1


Visualize the covariance and precision matrices

In [437]:
# fig, ax = plt.subplots(1,3,figsize=(20, 10))
# cov_x, _ = simulation.create_cov_prec_mat(omega)
# cov_x1, _ = simulation.create_cov_prec_mat(omega1)
# cov_x2, _ = simulation.create_cov_prec_mat(omega2)
# im0 = ax[0].imshow(np.abs(cov_x))
# im1 = ax[1].imshow(np.abs(cov_x1))
# im2 = ax[2].imshow(np.abs(cov_x2))
# im0.set_clim(0,1)
# im1.set_clim(0,1)
# im2.set_clim(0,1)

In [436]:
# _, ax = plt.subplots(1,3,figsize=(20, 10))
# im0 = ax[0].imshow(np.abs(omega))
# im1 = ax[1].imshow(np.abs(omega1))
# im2 = ax[2].imshow(np.abs(omega2))
# im0.set_clim(0,1)
# im1.set_clim(0,1)
# im2.set_clim(0,1)

In [93]:
# np.savez(f"temp/{f_out}_hub_bad_case.npz", omega=omega, omega1=omega1, omega2=omega2, g1_cov=g1_cov, g2_cov=g2_cov)

In [44]:
if 1:
    sim3_h5op.make_new_sim_data(
        dat_file,
        dat1,
        dat2,
        con_mat1,
        con_mat2,
        comm_gt=comm_gt,
        diff_gt=diff_gt,
        dep_mat_null=dep_mat_null,
        dep_mat_prior=dep_mat_prior,
        dep_mat_prior_loose=dep_mat_prior_loose,
    )
